In [1]:
using CSV, DataFrames, Dates, TimeZones

ENV["COLUMNS"] = 1000

1000

In [2]:
ID = "Charger01"

"Charger01"

In [3]:
# Read the CSV file from the specified path
df = CSV.read("data/charger_battery_data_$ID.csv", DataFrame)

# Filter the data
df = df[:, [:timestamp, :e_consumption, :e_production, :e_charger, :h_countdown, :soc_ev]]

df.e_consumption = df.e_consumption / 1000
df.e_production = df.e_production / 1000
df.e_charger = df.e_charger / 1000

# Parse the timestamps with timezone offset
timestamps = ZonedDateTime.(df.timestamp, DateFormat("yyyy-mm-dd HH:MM:SSzzzz"))

# Shift the timestamps one hour ahead
timestamps = timestamps .- Hour(1)

# Replace the timestamp column in the DataFrame with the shifted timestamps
df.timestamp = timestamps

35040-element Vector{ZonedDateTime}:
 2020-11-01T00:00:00+01:00
 2020-11-01T00:15:00+01:00
 2020-11-01T00:30:00+01:00
 2020-11-01T00:45:00+01:00
 2020-11-01T01:00:00+01:00
 2020-11-01T01:15:00+01:00
 2020-11-01T01:30:00+01:00
 2020-11-01T01:45:00+01:00
 2020-11-01T02:00:00+01:00
 2020-11-01T02:15:00+01:00
 ⋮
 2021-10-31T21:45:00+01:00
 2021-10-31T22:00:00+01:00
 2021-10-31T22:15:00+01:00
 2021-10-31T22:30:00+01:00
 2021-10-31T22:45:00+01:00
 2021-10-31T23:00:00+01:00
 2021-10-31T23:15:00+01:00
 2021-10-31T23:30:00+01:00
 2021-10-31T23:45:00+01:00

In [4]:
df

,timestamp,e_consumption,e_production,e_charger,h_countdown,soc_ev
,ZonedDa…,Float64,Float64,Float64?,Float64,Float64
1,2020-11-01T00:00:00+01:00,0.055,0.0,0.0,9.25,1.0
2,2020-11-01T00:15:00+01:00,0.037,0.0,0.0,9.0,1.0
3,2020-11-01T00:30:00+01:00,0.029,0.0,0.0,8.75,1.0
4,2020-11-01T00:45:00+01:00,0.029,0.0,0.0,8.5,1.0
5,2020-11-01T01:00:00+01:00,0.052,0.0,0.0,8.25,1.0
6,2020-11-01T01:15:00+01:00,0.037,0.0,0.0,8.0,1.0
7,2020-11-01T01:30:00+01:00,0.03,0.0,0.0,7.75,1.0
8,2020-11-01T01:45:00+01:00,0.034,0.0,0.0,7.5,1.0
9,2020-11-01T02:00:00+01:00,0.038,0.0,0.0,7.25,1.0


In [5]:
for col_name in names(df)
    display(("Column: ", col_name, ", Type: ", eltype(df[!, col_name])))
end

for col_name in names(df)
    missing_count = sum(ismissing.(df[!, col_name]))
    display(("Column: ", col_name, ", Missing Values: ", missing_count))
end

("Column: ", "timestamp", ", Type: ", ZonedDateTime)

("Column: ", "e_consumption", ", Type: ", Float64)

("Column: ", "e_production", ", Type: ", Float64)

("Column: ", "e_charger", ", Type: ", Union{Missing, Float64})

("Column: ", "h_countdown", ", Type: ", Float64)

...

("Column: ", "e_consumption", ", Missing Values: ", 0)

("Column: ", "e_production", ", Missing Values: ", 0)

("Column: ", "e_charger", ", Missing Values: ", 32475)

("Column: ", "h_countdown", ", Missing Values: ", 0)

("Column: ", "soc_ev", ", Missing Values: ", 0)

In [6]:
describe(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,timestamp,,2020-11-01T00:00:00+01:00,,2021-10-31T23:45:00+01:00,0,ZonedDateTime
2,e_consumption,0.078375,0.0,0.051,2.163,0,Float64
3,e_production,0.28957,0.0,0.0,2.325,0,Float64
4,e_charger,0.570734,0.0,0.0,2.51061,32475,"Union{Missing, Float64}"
5,h_countdown,0.536045,0.0,0.0,27.25,0,Float64
6,soc_ev,0.990868,0.0,1.0,1.0,0,Float64


In [7]:
# # Handle missing values
# total_missing = 0
# replaced_with_previous_day = 0
# replaced_with_zero = 0

# for col_name in names(df)
#     missing_indices = findall(ismissing, df[!, col_name])
#     total_missing += length(missing_indices)
    
#     for i in missing_indices
#         if i > 24
#             if ismissing(df[i-24, col_name])
#                 df[i, col_name] = 0
#                 replaced_with_zero += 1
#             else
#                 df[i, col_name] = df[i-24, col_name]
#                 replaced_with_previous_day += 1
#             end
#         else
#             df[i, col_name] = 0
#             replaced_with_zero += 1
#         end
#     end
# end

# println("Total missing values: ", total_missing)
# println("Replaced with previous day: ", replaced_with_previous_day)
# println("Replaced with zero: ", replaced_with_zero)

Resamplign from 15min intervalls to 1h intervalls. In the future, I'd like to try to work with 15min intervals.

In [8]:
function resample(df, time_column, interval)
    # Round the timestamps to the nearest hour
    df[!, time_column] = Dates.floor.(df[!, time_column], interval)
    
    # Define the columns to be summed
    sum_columns = ["e_consumption", "e_production", "e_charger"]
    
    # Group by the rounded timestamps and sum the other columns
    new_df = combine(groupby(df, time_column), sum_columns .=> (x -> sum(coalesce.(x, 0))) .=> sum_columns)
    
    # Handle "h_countdown" and "soc_ev" separately
    max_values = combine(groupby(df, time_column), "h_countdown" => maximum => "h_countdown")
    new_df = leftjoin(new_df, max_values, on=time_column)
    
    min_values = combine(groupby(df, time_column), "soc_ev" => minimum => "soc_ev")
    new_df = leftjoin(new_df, min_values, on=time_column)
    
    return new_df
end


resample (generic function with 1 method)

In [9]:
#=# Define a custom resampling function (LU)
function resample(df, time_column, interval)
    # Round the timestamps to the nearest hour
    df[!, time_column] = Dates.floor.(df[!, time_column], interval)
    
    # Group by the rounded timestamps and sum the other columns
    new_df = combine(groupby(df, time_column), names(df, Not(time_column)) .=> sum .=> names(df, Not(time_column)))
    
    return new_df
end

ErrorException: syntax: incomplete: unterminated multi-line comment #= ... =#

In [10]:
# Resample the data to 1-hour intervals
df_resampled = resample(df, :timestamp, Dates.Hour(1))

,timestamp,e_consumption,e_production,e_charger,h_countdown,soc_ev
,ZonedDa…,Float64,Float64,Float64,Float64?,Float64?
1,2020-11-01T00:00:00+01:00,0.15,0.0,0.0,9.25,1.0
2,2020-11-01T01:00:00+01:00,0.153,0.0,0.0,8.25,1.0
3,2020-11-01T02:00:00+01:00,0.151,0.0,0.0,7.25,1.0
4,2020-11-01T03:00:00+01:00,0.15,0.0,0.0,6.25,1.0
5,2020-11-01T04:00:00+01:00,0.145,0.0,0.0,5.25,1.0
6,2020-11-01T05:00:00+01:00,0.143,0.0,0.0,4.25,1.0
7,2020-11-01T06:00:00+01:00,0.16,0.0,0.0,3.25,1.0
8,2020-11-01T07:00:00+01:00,0.246,0.05,0.0,2.25,1.0
9,2020-11-01T08:00:00+01:00,0.245,0.099,0.0,1.25,1.0


In [38]:
# Change the type of the chargekwh column to Union{Missing, Float64}
df_resampled.e_charge = convert(Vector{Union{Missing, Float64}}, df_resampled.e_charger)


8760-element Vector{Union{Missing, Float64}}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [39]:
# Create a new DataFrame with the desired structure
new_df = DataFrame(
    electkwh = df_resampled.e_consumption,
    PV_generation = df_resampled.e_production,
    chargekwh = df_resampled.e_charger,
    h_countdown = df_resampled.h_countdown,
    soc_ev = df_resampled.soc_ev,
    month = month.(DateTime.(df_resampled.timestamp)),
    day = day.(DateTime.(df_resampled.timestamp)),
    hour = hour.(DateTime.(df_resampled.timestamp))
)

,electkwh,PV_generation,chargekwh,h_countdown,soc_ev,month,day,hour
,Float64,Float64,Float64,Float64?,Float64?,Int64,Int64,Int64
1,0.15,0.0,0.0,9.25,1.0,11,1,0
2,0.153,0.0,0.0,8.25,1.0,11,1,1
3,0.151,0.0,0.0,7.25,1.0,11,1,2
4,0.15,0.0,0.0,6.25,1.0,11,1,3
5,0.145,0.0,0.0,5.25,1.0,11,1,4
6,0.143,0.0,0.0,4.25,1.0,11,1,5
7,0.16,0.0,0.0,3.25,1.0,11,1,6
8,0.246,0.05,0.0,2.25,1.0,11,1,7
9,0.245,0.099,0.0,1.25,1.0,11,1,8


In [41]:

# Change the type of the chargekwh column to Union{Missing, Float64}
new_df.chargekwh = convert(Vector{Union{Missing, Float64}}, new_df.chargekwh)

# In every row where h_countdown is 0, set chargekwh to 'missing'
new_df[new_df.h_countdown .== 0, :chargekwh] .= missing

8074-element view(::Vector{Union{Missing, Float64}}, [11, 12, 13, 14, 15, 16, 17, 18, 19, 20  …  8751, 8752, 8753, 8754, 8755, 8756, 8757, 8758, 8759, 8760]) with eltype Union{Missing, Float64}:
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 ⋮
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing

In [42]:
new_df

,electkwh,PV_generation,chargekwh,h_countdown,soc_ev,month,day,hour
,Float64,Float64,Float64?,Float64?,Float64?,Int64,Int64,Int64
1,0.15,0.0,0.0,9.25,1.0,11,1,0
2,0.153,0.0,0.0,8.25,1.0,11,1,1
3,0.151,0.0,0.0,7.25,1.0,11,1,2
4,0.15,0.0,0.0,6.25,1.0,11,1,3
5,0.145,0.0,0.0,5.25,1.0,11,1,4
6,0.143,0.0,0.0,4.25,1.0,11,1,5
7,0.16,0.0,0.0,3.25,1.0,11,1,6
8,0.246,0.05,0.0,2.25,1.0,11,1,7
9,0.245,0.099,0.0,1.25,1.0,11,1,8


In [43]:


# Increase every value of h_countdown to the next higher integer
new_df.h_countdown = ceil.(new_df.h_countdown)

# For every row, where h_countdown has a value bigger than 0: 
# if the very next row has the same value in h_coundown, then increase the value by one
for i in 1:(nrow(new_df) - 1)
    if new_df[i, :h_countdown] > 0 && new_df[i, :h_countdown] == new_df[i + 1, :h_countdown]
        new_df[i, :h_countdown] += 1
    end
end


In [44]:
# Write the new DataFrame to a CSV file in the specified path
CSV.write("data/sonnen$(ID)_datafile_all.csv", new_df)

"data/sonnenCharger01_datafile_all.csv"

In [45]:
#Input_df = CSV.read("data/sonnen$(ID)_datafile_all.csv", DataFrame);

Input_df = new_df

,electkwh,PV_generation,chargekwh,h_countdown,soc_ev,month,day,hour
,Float64,Float64,Float64?,Float64,Float64?,Int64,Int64,Int64
1,0.15,0.0,0.0,10.0,1.0,11,1,0
2,0.153,0.0,0.0,9.0,1.0,11,1,1
3,0.151,0.0,0.0,8.0,1.0,11,1,2
4,0.15,0.0,0.0,7.0,1.0,11,1,3
5,0.145,0.0,0.0,6.0,1.0,11,1,4
6,0.143,0.0,0.0,5.0,1.0,11,1,5
7,0.16,0.0,0.0,4.0,1.0,11,1,6
8,0.246,0.05,0.0,3.0,1.0,11,1,7
9,0.245,0.099,0.0,2.0,1.0,11,1,8


In [46]:
describe(Input_df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,Type
1,electkwh,0.3135,0.076,0.219,7.182,0,Float64
2,PV_generation,1.15828,0.0,0.002,8.253,0,Float64
3,chargekwh,2.13401,0.0,0.925959,9.91823,8074,"Union{Missing, Float64}"
4,h_countdown,0.609817,0.0,0.0,28.0,0,Float64
5,soc_ev,0.988546,0.0,1.0,1.0,0,"Union{Missing, Float64}"
6,month,6.52683,1,7.0,12,0,Int64
7,day,15.7209,1,16.0,31,0,Int64
8,hour,11.5,0,11.5,23,0,Int64


In [47]:
# add new column with number of days
Input_df[!, :nday] = 1:nrow(Input_df)

1:8760

In [48]:
# calculate residual demand
Input_df[!, :d_res] = Input_df[!,:electkwh] + coalesce.(Input_df[!,:chargekwh], 0) - Input_df[!,:PV_generation]

8760-element Vector{Float64}:
  0.15
  0.153
  0.151
  0.15
  0.14500000000000002
  0.14300000000000002
  0.15999999999999998
  0.196
  0.146
  0.8710000000000002
  ⋮
 -0.187
  1.1420000000000001
  0.281
  0.28600000000000003
  0.248
  0.20400000000000001
  0.22299999999999998
  0.202
  0.174

## Add periodical time representation using cos/sin

In [49]:
# add columns with cos and sin values for periodical time values day + month
Input_df[!, :hour_cos] = cos.(Input_df[!,:hour] ./ maximum(Input_df[!,:hour]) .* 2*pi);
Input_df[!, :hour_sin] = sin.(Input_df[!,:hour] ./ maximum(Input_df[!,:hour]) .* 2*pi);

Input_df[!, :month_cos] = cos.(Input_df[!,:month] ./ maximum(Input_df[!,:month]) .* 2*pi);
Input_df[!, :month_sin] = sin.(Input_df[!,:month] ./ maximum(Input_df[!,:month]) .* 2*pi);

#Input_df[!, :nday_cos] = cos.(Input_df[!,:nday] ./ maximum(Input_df[!,:nday]) .* 2*pi);
#Input_df[!, :nday_sin] = sin.(Input_df[!,:nday] ./ maximum(Input_df[!,:nday]) .* 2*pi);

## Add seasons

In [50]:
Input_df[!, :spring] = (Input_df[!,:month] .>= 3) .* (Input_df[!,:month] .<= 5);
Input_df[!, :summer] = (Input_df[!,:month] .>= 6) .* (Input_df[!,:month] .<= 8);
Input_df[!, :autumn] = (Input_df[!,:month] .>= 9) .* (Input_df[!,:month] .<= 11);
Input_df[!, :winter] = convert.(Bool, (Input_df[!,:month] .>= 12) .+ (Input_df[!,:month] .<= 2));

Input_df[!, :season] = ifelse.(Input_df[!,:spring] .== true, 1,
                        ifelse.(Input_df[!,:summer] .== true, 2,
                        ifelse.(Input_df[!,:autumn] .== true, 3, 
                        4)));

In [51]:
describe(Input_df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,Type
1,electkwh,0.3135,0.076,0.219,7.182,0,Float64
2,PV_generation,1.15828,0.0,0.002,8.253,0,Float64
3,chargekwh,2.13401,0.0,0.925959,9.91823,8074,"Union{Missing, Float64}"
4,h_countdown,0.609817,0.0,0.0,28.0,0,Float64
5,soc_ev,0.988546,0.0,1.0,1.0,0,"Union{Missing, Float64}"
6,month,6.52683,1,7.0,12,0,Int64
7,day,15.7209,1,16.0,31,0,Int64
8,hour,11.5,0,11.5,23,0,Int64
9,nday,4380.5,1,4380.5,8760,0,Int64


## Add dynamic prices based on Ye et al. 2020

In [52]:
#LU
#=
function set_dynamic_prices(Input_df)
    map(eachrow(Input_df)) do r
        if r.month >= 5 && r.month <= 10
            if (r.hour >= 6 && r.hour <= 9) || (r.hour >= 16 && r.hour <= 17)
                return 0.3f0
            elseif (r.hour >= 10 && r.hour <= 15)
                return 0.6f0
            else
                return 0.15f0
            end
        elseif r.month >= 11 || r.month <= 4
            if (r.hour >= 6 && r.hour <= 9) || (r.hour >= 16 && r.hour <= 17)
                return 0.6f0
            elseif r.hour >= 10 && r.hour <= 15
                return 0.3f0
            else
                return 0.15f0
            end
        end
    end
end        

ErrorException: syntax: incomplete: unterminated multi-line comment #= ... =#

In [53]:
#LU
#=Input_df[!, "p_buy"] = set_dynamic_prices(Input_df);
Input_df[!, "p_sell"] = 0.5 .* Input_df[!, "p_buy"];

ErrorException: syntax: incomplete: unterminated multi-line comment #= ... =#

In [54]:
#LU describe(Input_df)

## Extract training, testing + evalution data set for summer, winter, both, all

In [55]:
#LU
#=# filter summer
Input_data_summer = filter(:summer => !=(0), Input_df)
describe(Input_data_summer), size(Input_data_summer)

ErrorException: syntax: incomplete: unterminated multi-line comment #= ... =#

In [56]:
function train_eval_test_split(Input_df)
    train = filter(row -> row.day <= 15, Input_df)
    eval = filter(row -> row.day > 15 && row.day <= 20, Input_df)
    test = filter(row -> row.day > 20, Input_df)
    return train, eval, test
end

train_eval_test_split (generic function with 1 method)

In [57]:
#LU
#=summer_training, summer_evaluation, summer_testing = train_eval_test_split(Input_data_summer)

CSV.write("data/$(ID)_summer_train_TOU.csv", summer_training);
CSV.write("data/$(ID)_summer_eval_TOU.csv", summer_evaluation);
CSV.write("data/$(ID)_summer_test_TOU.csv", summer_testing);

ErrorException: syntax: incomplete: unterminated multi-line comment #= ... =#

In [58]:
#LU
#=# filter winter
Input_data_winter = filter(:winter => !=(0), Input_df)
describe(Input_data_winter), size(Input_data_winter)

ErrorException: syntax: incomplete: unterminated multi-line comment #= ... =#

In [59]:
#LU
#=winter_training, winter_evaluation, winter_testing = train_eval_test_split(Input_data_winter)

# write data files
CSV.write("data/$(ID)_winter_train_TOU.csv", winter_training);
CSV.write("data/$(ID)_winter_eval_TOU.csv", winter_evaluation);
CSV.write("data/$(ID)_winter_test_TOU.csv", winter_testing);

ErrorException: syntax: incomplete: unterminated multi-line comment #= ... =#

In [60]:
all_training, all_evaluation, all_testing = train_eval_test_split(Input_df)

#LU 
#=
# write data files
CSV.write("data/$(ID)_all_train_TOU.csv", all_training);
CSV.write("data/$(ID)_all_eval_TOU.csv", all_evaluation);
CSV.write("data/$(ID)_all_test_TOU.csv", all_testing);

ErrorException: syntax: incomplete: unterminated multi-line comment #= ... =#

In [61]:
#LU
#=describe(vcat(Input_data_winter, Input_data_summer) ), size(vcat(Input_data_winter, Input_data_summer) )

ErrorException: syntax: incomplete: unterminated multi-line comment #= ... =#

In [62]:
#LU
#=# both seasons split
both_training, both_evaluation, both_testing = train_eval_test_split(vcat(Input_data_winter, Input_data_summer))

# write data files
CSV.write("data/$(ID)_both_train_TOU.csv", both_training);
CSV.write("data/$(ID)_both_test_TOU.csv", both_testing);
CSV.write("data/$(ID)_both_eval_TOU.csv", both_evaluation);

ErrorException: syntax: incomplete: unterminated multi-line comment #= ... =#

## Dataset for fixed prices

In [63]:
Input_df[!, "p_buy"] .= 0.3;
Input_df[!, "p_sell"] .= 0.1;

In [64]:
#LU
#=Input_data_summer = filter(:summer => !=(0), Input_df);
Input_data_winter = filter(:winter => !=(0), Input_df);

ErrorException: syntax: incomplete: unterminated multi-line comment #= ... =#

In [65]:
#LU summer_training, summer_evaluation, summer_testing = train_eval_test_split(Input_data_summer);
#LU winter_training, winter_evaluation, winter_testing = train_eval_test_split(Input_data_winter);
all_training, all_evaluation, all_testing = train_eval_test_split(Input_df);
#LU both_training, both_evaluation, both_testing = train_eval_test_split(vcat(Input_data_winter, Input_data_summer));

In [66]:
#LU
#=CSV.write("data/$(ID)_both_train_fix.csv", both_training);
CSV.write("data/$(ID)_both_test_fix.csv", both_testing);
CSV.write("data/$(ID)_both_eval_fix.csv", both_evaluation);=#

CSV.write("data/$(ID)_all_train_fix.csv", all_training);
CSV.write("data/$(ID)_all_test_fix.csv", all_testing);
CSV.write("data/$(ID)_all_eval_fix.csv", all_evaluation);

#LU
#=CSV.write("data/$(ID)_summer_train_fix.csv", summer_training);
CSV.write("data/$(ID)_summer_test_fix.csv", summer_testing);
CSV.write("data/$(ID)_summer_eval_fix.csv", summer_evaluation);
CSV.write("data/$(ID)_winter_train_fix.csv", winter_training);
CSV.write("data/$(ID)_winter_test_fix.csv", winter_testing);
CSV.write("data/$(ID)_winter_eval_fix.csv", winter_evaluation);

ErrorException: syntax: incomplete: unterminated multi-line comment #= ... =#